In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [3]:
brands = pd.read_csv('brands.csv')

In [4]:
brands.head()

,브랜드 이름,영어 이름,동의어/유의어,Unnamed: 3,Unnamed: 4,이슈 / 위험 요소
0,나이키,nike,NaN,NaN,NaN,"어뷰징, 콜라보 브랜드"
1,스톤아일랜드,stone island,NaN,NaN,NaN,"대리석집' 스톤아일랜드, 톰브라운 가품"
2,뉴발란스,new balance,NaN,NaN,NaN,NaN
3,아디다스,adidas,NaN,NaN,NaN,어뷰징
4,노스페이스,north face,NaN,NaN,NaN,NaN


In [5]:
brands = brands.iloc[:, :3]
brands.columns = ['name', 'eng_name', 'synonym']

In [6]:
for col in brands.columns:
    brands[col] = brands[col].astype(str).apply(lambda x: x.lower().strip())

In [99]:
brands2 = pd.read_csv('brands2.csv')
brands2 = brands2.iloc[:, :3]
brands2.columns = ['name', 'eng_name', 'synonym']
for col in brands2.columns:
    brands2[col] = brands2[col].astype(str).apply(lambda x: x.lower().strip())

In [8]:
q = '''
SELECT id, name, keyword, category_id
FROM service1_quicket.product_info
WHERE left(category_id, 3) IN ('310', '320', '400') AND status = 0
'''
products = pd.read_sql(q, con=bun_dw)

In [9]:
products.shape

(7897570, 4)

In [95]:
import re

def preprocess1(name):
    name = str(name)
    name = re.sub(r'\s+', ' ', name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  
    u"\U0001F300-\U0001F5FF"  
    u"\U0001F680-\U0001F6FF"  
    u"\U0001F1E0-\U0001F1FF"  
                       "]+", flags = re.UNICODE)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(' +', ' ', name) # ver 1
#     name = re.sub(' +', '', name) # ver 2
    return name.lower().strip()

In [96]:
preprocess_name = products.name.map(preprocess1)

In [97]:
import re

def preprocess2(name):
    name = str(name)
    name = re.sub(r'\s+', ' ', name)
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  
    u"\U0001F300-\U0001F5FF"  
    u"\U0001F680-\U0001F6FF"  
    u"\U0001F1E0-\U0001F1FF"  
                       "]+", flags = re.UNICODE)
    name = regrex_pattern.sub(r'',name)
    name = re.sub(r'[^\w\s]', ' ', name)
#     name = re.sub(' +', ' ', name) # ver 1
    name = re.sub(' +', '', name) # ver 2
    return name.lower().strip()

In [98]:
preprocess_name2 = products.name.map(preprocess2)

In [35]:
pids = products.id

In [15]:
preprocess_name[:10]

0                                      스판 티셔츠
1                                     슈프림 후드티
2                   스톤아일랜드 윈드브레이커 바람막이 6color
3    urbanic30 어바닉30 얼바닉30 스코틀랜드 해리스트위드 스커트입니
4                                루이비통 웨이드 스피디
5                                      네이비 자켓
6                                      팬디썬그라스
7                                      반전니트나시
8                                인기 lv 키폴 여행구
9                                      보테가베네타
Name: name, dtype: object

In [37]:
brand_dict = {key: [] for key in brands['name'].unique()}
for idx, row in brands.iterrows():
    eng = row.eng_name.split(',')
    syn = row.synonym.split(',')
    both = [i.strip() for i in [row['name']] + eng + syn if i != 'nan']
    brand_dict[row['name']] = both

In [102]:
brand_dict2 = {key: [] for key in brands2['name'].unique()}
for idx, row in brands2.iterrows():
    eng = row.eng_name.split(',')
    syn = row.synonym.split(',')
    both = [re.sub(' +', '', i).strip() for i in [row['name']] + eng + syn if i != 'nan']
    brand_dict2[row['name']] = both

In [38]:
brand_dict

{'mcm': ['mcm', '엠씨엠'],
 '겐조': ['겐조', 'kenzo'],
 '고야드': ['고야드', 'goyard'],
 '골든구스': ['골든구스', 'golden goose'],
 '골든듀': ['골든듀'],
 '구찌': ['구찌', 'gucci'],
 '구호': ['구호', 'kuho'],
 '그라미치': ['그라미치', 'gramicci'],
 '그랜드세이코': ['그랜드세이코', 'grand seiko', '세이코'],
 '그레고리': ['그레고리', 'gregory'],
 '까르띠에': ['까르띠에', 'cartier'],
 '까스텔바작': ['까스텔바작', 'castel bajac'],
 '꼼데가르송': ['꼼데가르송', 'comme des garcons'],
 '끌로에': ['끌로에', 'chloe'],
 '나나미카': ['나나미카', 'nanamica'],
 '나이젤카본': ['나이젤카본', 'nigel cabourn'],
 '나이키': ['나이키', 'nike'],
 '낫띵리튼': ['낫띵리튼', 'nothing written'],
 '내셔널지오그래픽': ['내셔널지오그래픽',
  'national geographic',
  '네셔널지오그래픽',
  '내셔널지오그래피',
  '네셔널지오그래피'],
 '널디': ['널디', 'nerdy'],
 '넘버나인': ['넘버나인', 'number (n)ine'],
 '네이더스': ['네이더스', 'neithers'],
 '노스페이스': ['노스페이스', 'north face'],
 '뉴발란스': ['뉴발란스', 'new balance'],
 '뉴에라': ['뉴에라', 'new era'],
 '느와르라르메스': ['느와르라르메스', 'noir larmes'],
 '니들스': ['니들스', 'needles'],
 '닐바렛': ['닐바렛', 'neil barrett'],
 '다미아니': ['다미아니', 'damiani'],
 '다이나핏': ['다이나핏', 'dynafit'],
 '다이애그널': 

In [103]:
brand_dict2

{'mcm': ['mcm', '엠씨엠'],
 '겐조': ['겐조', 'kenzo'],
 '고야드': ['고야드', 'goyard'],
 '골든구스': ['골든구스', 'goldengoose'],
 '골든듀': ['골든듀'],
 '구찌': ['구찌', 'gucci'],
 '구호': ['구호', 'kuho'],
 '그라미치': ['그라미치', 'gramicci'],
 '그랜드세이코': ['그랜드세이코', 'grandseiko', '세이코'],
 '그레고리': ['그레고리', 'gregory'],
 '까르띠에': ['까르띠에', 'cartier'],
 '까스텔바작': ['까스텔바작', 'castelbajac'],
 '꼼데가르송': ['꼼데가르송', 'commedesgarcons'],
 '끌로에': ['끌로에', 'chloe'],
 '나나미카': ['나나미카', 'nanamica'],
 '나이젤카본': ['나이젤카본', 'nigelcabourn'],
 '나이키': ['나이키', 'nike', '조던'],
 '낫띵리튼': ['낫띵리튼', 'nothingwritten'],
 '내셔널지오그래픽': ['내셔널지오그래픽',
  'nationalgeographic',
  '네셔널지오그래픽',
  '내셔널지오그래피',
  '네셔널지오그래피'],
 '널디': ['널디', 'nerdy'],
 '넘버나인': ['넘버나인', 'number(n)ine'],
 '네이더스': ['네이더스', 'neithers'],
 '노스페이스': ['노스페이스', 'northface'],
 '뉴발란스': ['뉴발란스', 'newbalance', '뉴발'],
 '뉴에라': ['뉴에라', 'newera'],
 '느와르라르메스': ['느와르라르메스', 'noirlarmes'],
 '니들스': ['니들스', 'needles'],
 '닐바렛': ['닐바렛', 'neilbarrett'],
 '다미아니': ['다미아니', 'damiani'],
 '다이나핏': ['다이나핏', 'dynafit'],
 '다이애그널': ['

- preprocess_name: a list of preprocessed product names
- brand_dict: {brand name: list of brand name in english and synonyms}
- name_match: {idx of preprocess_name: list of brands that match the product name}

In [56]:
name_match = {}
for idx, name in enumerate(preprocess_name):
    for brand, word_list in brand_dict.items():
        for word in word_list:
            if word in name:
                brands = name_match.get(idx, [])
                if brand not in brands:
                    brands.append(brand)
                    name_match[idx] = brands

In [57]:
len(name_match)

2580064

In [58]:
import json
name_match_json = json.dumps(name_match)

In [59]:
with open('name_match.txt', 'w') as outfile:
    json.dump(name_match_json, outfile)

In [60]:
names = name_match.keys()

In [61]:
list(names)[:10]

[1, 2, 4, 9, 14, 15, 21, 23, 26, 28]

In [65]:
tmp_list = [1, 2, 4, 9, 14, 15, 21, 23, 26, 28]
for i in tmp_list:
    print(preprocess_name[i], name_match[i])

슈프림 후드티 ['슈프림']
스톤아일랜드 윈드브레이커 바람막이 6color ['스톤아일랜드']
루이비통 웨이드 스피디 ['루이비통']
보테가베네타 ['보테가베네타']
양모 버버리 워커힐 워너비 ['버버리']
샤넬귀걸이 ['샤넬']
폴스미스 봄 가을 곤색 면바지 남성 34 k12 314 ['폴스미스']
당일배송가능 260 280 구찌 모던 로퍼 ['구찌']
에르메스 오즈 켈리 뮬 로퍼 6color ['에르메스']
에르메스st 은침귀걸이 ['에르메스']


In [67]:
no = []
for i in range(len(preprocess_name)):
    if i not in names:
        no.append(i)

In [68]:
len(no)

5317506

In [73]:
f=open('not_filtered_name.txt','w')
for i in no:
    n = preprocess_name[i]
    f.write(n+'\n')

In [90]:
a = no[3960]
products.loc[a]

id                                 99845568
name                                  몽클 맨투
keyword        몽클레어,몽클레어맨투맨,남자맨투맨,몽클맨투맨,맨투맨
category_id                       320030100
Name: 5467, dtype: object

In [105]:
# 하나의 상품 이름에 두개 이상의 브랜드가 속하는 경우
duplicates = []
for k, v in name_match.items():
    if len(v) > 1:
        duplicates.append(k)

In [110]:
len(duplicates)/len(name_match)

0.056461390105051655

In [111]:
name_match2 = {}
for idx, name in enumerate(preprocess_name2):
    for brand, word_list in brand_dict2.items():
        for word in word_list:
            if word in name:
                brands = name_match2.get(idx, [])
                if brand not in brands:
                    brands.append(brand)
                    name_match2[idx] = brands

In [118]:
len(name_match2)

2684895

In [115]:
(len(name_match2)-len(name_match))/len(name_match)

0.04063116263782604

In [116]:
duplicates2 = []
for k, v in name_match2.items():
    if len(v) > 1:
        duplicates2.append(k)
print(len(duplicates2))

156022


In [117]:
len(duplicates2)/len(name_match2)

0.05811102482592429

In [119]:
f=open('name_match2.txt','w')
cnt = 0
for i in name_match2:
    cnt += 1
    n = preprocess_name2[i]
    f.write(n + ' || ' + str(name_match2[i]) + '\n')
    if cnt == 10000:
        break

In [127]:
preprocess_name2.tolist().index('나이키피오갓250')

25159

In [128]:
preprocess_name[25159]

'나이키 피오갓 250'

In [129]:
pids[25159]

101107652

In [132]:
get_brand = []
for k, v in name_match2.items():
    get_brand += v
get_brand = list(set(get_brand))

In [133]:
len(get_brand)

332